<a href="https://colab.research.google.com/github/dgaisan/ai-agents/blob/main/CrewAI-based-Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Running Agents using Ollama and CrewAI**

Step 1: Environment Setup


Installs ollama dependency

In [ ]:
%pip install ollama
%pip install colab-xterm

Installs utilities for checking CPU/GPU avail

In [ ]:
!sudo apt-get update
!sudo apt-get install pciutils lshw

Downloads and installs Ollama via the official shell script (Run only once)

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

Pulling Deepseek & llama Models

In [ ]:
# Download the deepseek-r1:7b
!ollama pull deepseek-r1:7b
# Download the llama3
!ollama pull llama3

Pulling Lightweight SLMs

In [ ]:
# ultra-small general models
!ollama pull phi3:mini
!ollama pull tinyllama
# Google's compact chat model
!ollama pull gemma:2b
# reasoning model
!ollama pull deepseek-r1:1.5b

Check Installed Models

In [ ]:
!ollama list

Step 2: Launching Ollama server

Imports libs for managing HTTP requests, subprocesses, and multithreading

In [ ]:
import json
import requests
import subprocess
import threading
from pprint import pprint

Launches Ollama server in the background

In [ ]:
# Spin up Ollama Server
def run_ollama():
  subprocess.Popen(["ollama", "serve"])
thread = threading.Thread(target=run_ollama)
thread.start()

Tests if the server is running

In [ ]:
!curl http://127.0.0.1:11434

Silences some warning (optional)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# avoids restart session warning in Colab
# import sys
# sys.modules.pop('PIL', None)

Step 3: CrewAI integration

Agents are defined using CrewAI.
Enables multi-agent workflows.
Agents are connected to Ollama server

Installs CrewAI libs/tools

In [ ]:
%pip install git+https://github.com/joaomdmoura/crewAI.git --quiet
%pip install crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere --quiet
print("---")
%pip show crewAI crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere
%pip install -qU langchain-ollama

Agent imports dependencies

In [ ]:
from crewai import Agent, Task, Crew, Process
from getpass import getpass
from langchain_ollama import ChatOllama
from textwrap import dedent
import os

Define Agents with CrewAI

In [ ]:
from crewai import Agent
from textwrap import dedent
from langchain_ollama import ChatOllama
from crewai import LLM

MODEL_NAME = "ollama/tinyllama:latest"
OLLAMA_ENDPOINT = "http://127.0.0.1:11434"

llm = LLM(model=MODEL_NAME, base_url=OLLAMA_ENDPOINT)

# Agent 1: Cloud Infrastructure Planner
# Generates infrastructure plans (e.g., recommends services, architectures, scalability strategies)
agent_1 = Agent(
    role=dedent("""Cloud Infrastructure Planner"""),
    goal=dedent("""Design efficient, scalable, and reliable cloud infrastructure based on project requirements."""),
    backstory=dedent("""You are a cloud architecture expert with deep knowledge of AWS, Azure, and other major platforms. You excel at designing robust, cost-effective infrastructure tailored to technical and business needs."""),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)

# Agent 2: Security Auditor
# Reviews plans/configs to find risks, misconfigurations, compliance gaps
agent_2 = Agent(
    role=dedent("""Security Auditor"""),
    goal=dedent("""Assess cloud infrastructure designs and configurations to identify potential security risks and recommend improvements."""),
    backstory=dedent("""You are a cybersecurity specialist focused on cloud environments. Your expertise lies in spotting vulnerabilities, enforcing best practices, and ensuring systems are secure by design."""),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)


Define Tasks for Agents (This is what Agents will be trying to achieve)

In [ ]:
from crewai import Task
from textwrap import dedent

# Task for Cloud Infrastructure Planner
task_1 = Task(
    description=dedent("""
        Design a scalable, cost-efficient cloud infrastructure for a web application that expects moderate traffic (~10,000 daily active users) with occasional traffic spikes. The application requires high availability, data storage, and basic monitoring.
    """),
    expected_output=dedent("""
        A high-level infrastructure plan (~200 words) that includes:
        - Recommended cloud provider and reasoning
        - Core services to be used (e.g., compute, storage, load balancing)
        - Scalability and high availability considerations
        - Optional: Cost optimization suggestions
    """),
    agent=agent_1,
)

# Task for Security Auditor
task_2 = Task(
    description=dedent("""
        Review the proposed cloud infrastructure for the web application and identify potential security risks. Focus on aspects like data protection, access control, and network security. Suggest practical improvements or mitigations.
    """),
    expected_output=dedent("""
        A concise security audit (~200 words) that covers:
        - Key security risks or gaps in the proposed design
        - Recommended security best practices for the identified risks
        - Optional: Notes on compliance considerations (e.g., GDPR, SOC 2)
    """),
    agent=agent_2,
    context=[task_1],
)

In [ ]:
print('-------------------------------------------')

topic_input = input("🔍 Enter the Topic to explore\n")
format_input = input("🧠 What output format are you intersted in (e.g., article, summary)?\n")

print('-------------------------------------------')
print(f"✔️ Topic Input: {topic_input}\nFormat: {format_input}\n")


Kicks in CrewAI work

In [ ]:
def main():
    crew = Crew(
        agents=[agent_1, agent_2],
        tasks=[task_1, task_2],
        verbose=True,
        process=Process.sequential, # (e.g., sequential, hierarchical)
        planning_llm=llm
    )

    inputs = {
    "topic": topic_input,
    "format": format_input
    }

    result = crew.kickoff(inputs=inputs)
    print("\n\n-----------------------------------")
    print(result)

    return result

if __name__ == "__main__":
  result = main()

In [ ]:
from IPython.display import display, Markdown

markdown_text = result.raw  # Adjust this based on the actual attribute

display(Markdown(markdown_text))